In [25]:
import requests
import json
import pandas as pd
import numpy as np
import datetime
import re

import string
import pickle
import os.path
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
pd.set_option('display.float_format',lambda x : '%.4f' % x)

In [26]:
#時間日期設定
ddd=(datetime.datetime.now() - datetime.timedelta(days=1)).strftime("%Y%m%d") 
ddd2=(datetime.datetime.now() - datetime.timedelta(days=1)).strftime("%Y-%m-%d") 
#downloadpath='//navdi/DEM_UserData/btorin/Desktop/download_csv/'

In [34]:
#抓出線下所有數據
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.89 Safari/537.36'}
payload = {"header":{"username": "newpopo","password": "Po774949","token": "84f617e63137ccc5cf246f4ac6a1d3b8","account_type": '1'}}
session_requests = requests.session()
response_baidu_list = session_requests.request('post',url= "https://api.baidu.com/json/tongji/v1/ReportService/getSiteList" , data=json.dumps(payload),headers=headers)
response_baidu_list=response_baidu_list.json()['body']['data'][0]['list']
response_baidu_list=pd.DataFrame(response_baidu_list)[['create_time','domain','site_id']]

In [35]:
response_baidu_list=response_baidu_list.iloc[[1]]
response_baidu_list.reset_index(inplace=True, drop=True)

In [63]:
df_pd_data_url=pd.DataFrame(columns=['pageId','name','PV','UV','ip_count','AV','domain'])
for j in range(len(response_baidu_list)):
    payload = {"header":{"username": "newpopo","password": "Po774949","token": "84f617e63137ccc5cf246f4ac6a1d3b8","account_type": '1'},
              "body": {"site_id": str(response_baidu_list.loc[j,'site_id']),"start_date": ddd,"end_date": ddd,"metrics": "pv_count,visitor_count,ip_count,average_stay_time","method": "visit/toppage/a","max_results": "1000"}}
    response_baidu_url = session_requests.request('post',url= "https://api.baidu.com/json/tongji/v1/ReportService/getData" , data=json.dumps(payload),headers=headers)
    response_baidu_url=response_baidu_url.json()['body']['data'][0]['result']['items']
    #scrape diff data
    pd_data_url=pd.DataFrame(columns=['pageId','name'])
    for i in range(len(response_baidu_url[0])) :
        name_data=pd.DataFrame(response_baidu_url[0][i])[['pageId','name']]
        pd_data_url=pd_data_url.append(name_data)
    pd_data_url=pd_data_url.reset_index(drop=True)
    pd_data_url[['PV','UV','ip_count','AV']]=pd.DataFrame(response_baidu_url[1],columns=['PV','UV','ip_count','AV'])
    pd_data_url['domain']=response_baidu_list.loc[j,'domain']
    df_pd_data_url=df_pd_data_url.append(pd_data_url)
    df_pd_data_url=df_pd_data_url.reset_index(drop=True)

In [64]:
df_pd_data_url['date']=ddd2
df_pd_data_url['domain_sub']=[x.split('.com')[0]+str('.com') for x in df_pd_data_url['name'] ]
final_data=df_pd_data_url[['date','domain','domain_sub','name','PV','UV','ip_count','AV']]

In [66]:
# If modifying these scopes, delete the file token.pickle.
SCOPES = ['https://www.googleapis.com/auth/analytics.readonly',
      'https://www.googleapis.com/auth/drive',
      'https://www.googleapis.com/auth/spreadsheets']
creds = None
if os.path.exists('token.pickle'):
    with open('token.pickle', 'rb') as token:
        creds = pickle.load(token)
# If there are no (valid) credentials available, let the user log in.
if not creds or not creds.valid:
    if creds and creds.expired and creds.refresh_token:
        creds.refresh(Request())
    else:
        flow = InstalledAppFlow.from_client_secrets_file(
            'credentials.json', SCOPES)
        creds = flow.run_local_server(port=0)
    # Save the credentials for the next run
    with open('token.pickle', 'wb') as token:
        pickle.dump(creds, token)
service = build('sheets', 'v4', credentials=creds)
# Call the Sheets API
sheet = service.spreadsheets()

#寫入 data 進入 google sheet #美東時間
def insert_data(value_range_body,sheetname,colname1,colname2,col1,col2) :
    SAMPLE_RANGE_NAME = str(sheetname)+'!'+colname1+str(col1)+':'+colname2+str(col2)
    request= sheet.values().update(spreadsheetId=SAMPLE_SPREADSHEET_ID,valueInputOption='USER_ENTERED',range=SAMPLE_RANGE_NAME , body=value_range_body).execute()
def google_sheet_insert_data(sheet_name,data,SAMPLE_SPREADSHEET_ID) : 
    #查詢目前使用到維度
    SAMPLE_RANGE_NAME = sheet_name+'!A1:A70000'
    result_index = sheet.values().get(spreadsheetId=SAMPLE_SPREADSHEET_ID, range=SAMPLE_RANGE_NAME).execute()
    insert_row=len(result_index.get('values'))+1
    #遊戲驗證
    value_range_body = {"majorDimension":"ROWS","values": data.values.tolist() }
    insert_data(value_range_body,sheet_name,'A','H',insert_row,insert_row+1000)
    
#Boin data insert
SAMPLE_SPREADSHEET_ID = '1ju3TV2MrFW8_xUSvIt62YbXNc7n5JkodAO3ATT1VdD0'
google_sheet_insert_data('豆豆百度数据',final_data,SAMPLE_SPREADSHEET_ID)

In [ ]:
#df_pd_data_search=pd.DataFrame(columns=['keywordId','name','PV','UV','domain'])
#for j in range(len(response_baidu_list)):
#    payload = {"header":{"username": "newpopo","password": "Po774949","token": "84f617e63137ccc5cf246f4ac6a1d3b8","account_type": '1'},
#              "body": {"site_id": str(response_baidu_list.loc[j,'site_id']),"start_date": ddd,"end_date": ddd,"metrics": "pv_count,visitor_count","method": "source/searchword/a"}}
#    response_baidu_searchword = session_requests.request('post',url= "https://api.baidu.com/json/tongji/v1/ReportService/getData" , data=json.dumps(payload),headers=headers)
#    response_baidu_searchword=response_baidu_searchword.json()['body']['data'][0]['result']['items']
#    #scrape diff data
#    pd_data_search=pd.DataFrame(columns=['keywordId','name'])
#    for i in range(len(response_baidu_searchword[0])) :
#        name_data=pd.DataFrame(response_baidu_searchword[0][i])[['keywordId','name']]
#        pd_data_search=pd_data_search.append(name_data)
#    pd_data_search=pd_data_search.reset_index(drop=True)
#    pd_data_search[['PV','UV']]=pd.DataFrame(response_baidu_searchword[1],columns=['PV','UV'])
#    pd_data_search['domain']=response_baidu_list.loc[j,'domain']
#    df_pd_data_search=df_pd_data_search.append(pd_data_search)
#    df_pd_data_search=df_pd_data_search.reset_index(drop=True)
#df_pd_data_search[df_pd_data_search['PV'] >5].sort_values(by=['UV'],ascending=False)